In [2]:
import pandas as pd
import nibabel as nib
import numpy as np
from os import path

df = pd.read_csv('../Exploration/demographics.csv')

output_data = '/data/3d/fcp-data.pt'
output_labels = '/data/3d/fcp-labels.pt'

df = df[df.type == 'fcp']
df

,Unnamed: 0,subject,serial_no,age,sex,hospital,group,processed_data_path,type
0,0,sub03951,1,37.32,m,NewYork_a_ADHD,NaN,/data-ixi-fcp/fcp/NewYork_a_ADHD/sub03951,fcp
1,1,sub08595,2,23.38,m,NewYork_a_ADHD,NaN,/data-ixi-fcp/fcp/NewYork_a_ADHD/sub08595,fcp
2,2,sub12486,3,40.77,m,NewYork_a_ADHD,NaN,/data-ixi-fcp/fcp/NewYork_a_ADHD/sub12486,fcp
3,3,sub14299,4,24.58,f,NewYork_a_ADHD,NaN,/data-ixi-fcp/fcp/NewYork_a_ADHD/sub14299,fcp
4,4,sub14465,5,47.06,m,NewYork_a_ADHD,NaN,/data-ixi-fcp/fcp/NewYork_a_ADHD/sub14465,fcp
5,5,sub15758,6,32.84,m,NewYork_a_ADHD,NaN,/data-ixi-fcp/fcp/NewYork_a_ADHD/sub15758,fcp
6,6,sub17109,7,46.15,m,NewYork_a_ADHD,NaN,/data-ixi-fcp/fcp/NewYork_a_ADHD/sub17109,fcp
7,7,sub20676,8,49.19,m,NewYork_a_ADHD,NaN,/data-ixi-fcp/fcp/NewYork_a_ADHD/sub20676,fcp
8,8,sub20691,9,26.35,m,NewYork_a_ADHD,NaN,/data-ixi-fcp/fcp/NewYork_a_ADHD/sub20691,fcp
9,9,sub22349,10,26.15,m,NewYork_a_ADHD,NaN,/data-ixi-fcp/fcp/NewYork_a_ADHD/sub22349,fcp


In [3]:
skullstripped_images_adni = []
files_not_found = []
labels = []
for index, row in df.iterrows():
    if row['type'] == 'fcp':
        print('Processing FCP type dataset')
        if path.exists(row['processed_data_path'] + '/anat/mprage_skullstripped.nii.gz'):
            print(f'Processing {index + 1}')
            print(row['processed_data_path'])
            skullstripped_img = nib.load(row['processed_data_path'] + '/anat/mprage_skullstripped.nii.gz').get_data()
            skullstripped_img = (skullstripped_img - np.mean(skullstripped_img)) / np.std(skullstripped_img)
            print(f'Shape: {skullstripped_img.shape}')
            skullstripped_images_adni.append(skullstripped_img)
            print(row['age'])
            labels.append(row['age'])
        else:
            files_not_found.append(row['processed_data_path'])
    else:
        if path.exists(row['processed_data_path'] + '/T1_biascorr.nii.gz') and path.exists(row['processed_data_path'] + '/T1_fast_seg.nii.gz'):
            print(f'Processing {index + 1}')
            print(row['processed_data_path'])
            orig_img = nib.load(row['processed_data_path'] + '/T1_biascorr.nii.gz').get_data()
            seg_img = nib.load(row['processed_data_path'] + '/T1_fast_seg.nii.gz').get_data()
            skullstripped_img = orig_img * (seg_img > 0)
            skullstripped_img = (skullstripped_img - np.mean(skullstripped_img)) / np.std(skullstripped_img)
            print(f'Shape: {skullstripped_img.shape}')
            skullstripped_images_adni.append(skullstripped_img)
            print(row['age'])
            labels.append(row['age'])
        else:
            files_not_found.append(row['processed_data_path'])

Processing FCP type dataset
Processing 1
/data-ixi-fcp/fcp/NewYork_a_ADHD/sub03951
Shape: (176, 256, 256)
37.32
Processing FCP type dataset
Processing 2
/data-ixi-fcp/fcp/NewYork_a_ADHD/sub08595
Shape: (176, 256, 256)
23.38
Processing FCP type dataset
Processing 3
/data-ixi-fcp/fcp/NewYork_a_ADHD/sub12486
Shape: (176, 256, 256)
40.77
Processing FCP type dataset
Processing 4
/data-ixi-fcp/fcp/NewYork_a_ADHD/sub14299
Shape: (176, 256, 256)
24.58
Processing FCP type dataset
Processing 5
/data-ixi-fcp/fcp/NewYork_a_ADHD/sub14465
Shape: (176, 256, 256)
47.06
Processing FCP type dataset
Processing 6
/data-ixi-fcp/fcp/NewYork_a_ADHD/sub15758
Shape: (176, 256, 256)
32.84
Processing FCP type dataset
Processing 7
/data-ixi-fcp/fcp/NewYork_a_ADHD/sub17109
Shape: (176, 256, 256)
46.15
Processing FCP type dataset
Processing 8
/data-ixi-fcp/fcp/NewYork_a_ADHD/sub20676
Shape: (176, 256, 256)
49.19
Processing FCP type dataset
Processing 9
/data-ixi-fcp/fcp/NewYork_a_ADHD/sub20691
Shape: (176, 256, 256

Shape: (176, 256, 256)
22.0
Processing FCP type dataset
Processing 80
/data-ixi-fcp/fcp/ICBM/sub65921
Shape: (160, 256, 256)
20.0
Processing FCP type dataset
Processing 81
/data-ixi-fcp/fcp/ICBM/sub66085
Shape: (176, 256, 256)
30.0
Processing FCP type dataset
Processing 82
/data-ixi-fcp/fcp/ICBM/sub66794
Shape: (160, 256, 256)
20.0
Processing FCP type dataset
Processing 83
/data-ixi-fcp/fcp/ICBM/sub68850
Shape: (176, 256, 256)
70.0
Processing FCP type dataset
Processing 84
/data-ixi-fcp/fcp/ICBM/sub70595
Shape: (176, 256, 256)
44.0
Processing FCP type dataset
Processing 85
/data-ixi-fcp/fcp/ICBM/sub71932
Shape: (176, 256, 256)
21.0
Processing FCP type dataset
Processing 86
/data-ixi-fcp/fcp/ICBM/sub72135
Shape: (176, 256, 256)
43.0
Processing FCP type dataset
Processing 87
/data-ixi-fcp/fcp/ICBM/sub73490
Shape: (176, 256, 256)
68.0
Processing FCP type dataset
Processing 88
/data-ixi-fcp/fcp/ICBM/sub76325
Shape: (176, 256, 256)
57.0
Processing FCP type dataset
Processing 89
/data-ixi-fc

Shape: (256, 256, 166)
22.0
Processing FCP type dataset
Processing 160
/data-ixi-fcp/fcp/Oulu/sub23392
Shape: (256, 256, 166)
22.0
Processing FCP type dataset
Processing 161
/data-ixi-fcp/fcp/Oulu/sub24781
Shape: (256, 256, 170)
22.0
Processing FCP type dataset
Processing 162
/data-ixi-fcp/fcp/Oulu/sub25331
Shape: (256, 256, 166)
22.0
Processing FCP type dataset
Processing 163
/data-ixi-fcp/fcp/Oulu/sub26554
Shape: (256, 256, 166)
22.0
Processing FCP type dataset
Processing 164
/data-ixi-fcp/fcp/Oulu/sub27274
Shape: (256, 256, 166)
21.0
Processing FCP type dataset
Processing 165
/data-ixi-fcp/fcp/Oulu/sub27392
Shape: (256, 256, 166)
22.0
Processing FCP type dataset
Processing 166
/data-ixi-fcp/fcp/Oulu/sub29793
Shape: (256, 256, 172)
22.0
Processing FCP type dataset
Processing 167
/data-ixi-fcp/fcp/Oulu/sub31444
Shape: (256, 256, 166)
22.0
Processing FCP type dataset
Processing 168
/data-ixi-fcp/fcp/Oulu/sub32052
Shape: (256, 256, 158)
22.0
Processing FCP type dataset
Processing 169
/d

Shape: (256, 256, 140)
20.0
Processing FCP type dataset
Processing 240
/data-ixi-fcp/fcp/Leiden_2180/sub28473
Shape: (256, 256, 140)
23.0
Processing FCP type dataset
Processing 241
/data-ixi-fcp/fcp/Leiden_2180/sub38454
Shape: (256, 256, 140)
21.0
Processing FCP type dataset
Processing 242
/data-ixi-fcp/fcp/Leiden_2180/sub52853
Shape: (256, 256, 140)
27.0
Processing FCP type dataset
Processing 243
/data-ixi-fcp/fcp/Leiden_2180/sub56299
Shape: (256, 256, 140)
21.0
Processing FCP type dataset
Processing 244
/data-ixi-fcp/fcp/Leiden_2180/sub58194
Shape: (256, 256, 140)
27.0
Processing FCP type dataset
Processing 245
/data-ixi-fcp/fcp/Leiden_2180/sub99856
Shape: (256, 256, 140)
24.0
Processing FCP type dataset
Processing 246
/data-ixi-fcp/fcp/AnnArbor_a/sub04111
Shape: (106, 256, 256)
25.63
Processing FCP type dataset
Processing 247
/data-ixi-fcp/fcp/AnnArbor_a/sub04619
Shape: (106, 256, 256)
18.34
Processing FCP type dataset
Processing 248
/data-ixi-fcp/fcp/AnnArbor_a/sub13636
Shape: (106

Shape: (176, 256, 256)
45.83
Processing FCP type dataset
Processing 317
/data-ixi-fcp/fcp/NewYork_a/sub66085
Shape: (176, 256, 256)
24.15
Processing FCP type dataset
Processing 318
/data-ixi-fcp/fcp/NewYork_a/sub66794
Shape: (176, 256, 256)
23.72
Processing FCP type dataset
Processing 319
/data-ixi-fcp/fcp/NewYork_a/sub70566
Shape: (176, 256, 256)
25.24
Processing FCP type dataset
Processing 320
/data-ixi-fcp/fcp/NewYork_a/sub80566
Shape: (176, 256, 256)
26.33
Processing FCP type dataset
Processing 321
/data-ixi-fcp/fcp/NewYork_a/sub82228
Shape: (176, 256, 256)
35.0
Processing FCP type dataset
Processing 322
/data-ixi-fcp/fcp/NewYork_a/sub82716
Shape: (176, 256, 256)
30.14
Processing FCP type dataset
Processing 323
/data-ixi-fcp/fcp/NewYork_a/sub82754
Shape: (176, 256, 256)
23.06
Processing FCP type dataset
Processing 324
/data-ixi-fcp/fcp/NewYork_a/sub83782
Shape: (176, 256, 256)
25.98
Processing FCP type dataset
Processing 325
/data-ixi-fcp/fcp/NewYork_a/sub83978
Shape: (176, 256, 25

...

Shape: (176, 224, 256)
23.0
Processing FCP type dataset
Processing 686
/data-ixi-fcp/fcp/Atlanta/sub60499
Shape: (176, 240, 256)
29.0
Processing FCP type dataset
Processing 687
/data-ixi-fcp/fcp/Atlanta/sub61442
Shape: (176, 224, 256)
29.0
Processing FCP type dataset
Processing 688
/data-ixi-fcp/fcp/Atlanta/sub61902
Shape: (192, 240, 256)
23.0
Processing FCP type dataset
Processing 689
/data-ixi-fcp/fcp/Atlanta/sub71337
Shape: (176, 240, 256)
27.0
Processing FCP type dataset
Processing 690
/data-ixi-fcp/fcp/Atlanta/sub72096
Shape: (176, 240, 256)
23.0
Processing FCP type dataset
Processing 691
/data-ixi-fcp/fcp/Atlanta/sub72971
Shape: (176, 240, 256)
30.0
Processing FCP type dataset
Processing 692
/data-ixi-fcp/fcp/Atlanta/sub75153
Shape: (176, 224, 256)
32.0
Processing FCP type dataset
Processing 693
/data-ixi-fcp/fcp/Atlanta/sub76280
Shape: (176, 224, 256)
23.0
Processing FCP type dataset
Processing 694
/data-ixi-fcp/fcp/Atlanta/sub81596
Shape: (192, 240, 256)
53.0
Processing FCP typ

Shape: (128, 212, 169)
24.0
Processing FCP type dataset
Processing 762
/data-ixi-fcp/fcp/Beijing_Zang/sub08455
Shape: (128, 191, 185)
25.0
Processing FCP type dataset
Processing 763
/data-ixi-fcp/fcp/Beijing_Zang/sub08816
Shape: (128, 183, 197)
21.0
Processing FCP type dataset
Processing 764
/data-ixi-fcp/fcp/Beijing_Zang/sub08992
Shape: (128, 200, 173)
20.0
Processing FCP type dataset
Processing 765
/data-ixi-fcp/fcp/Beijing_Zang/sub10186
Shape: (128, 200, 167)
19.0
Processing FCP type dataset
Processing 766
/data-ixi-fcp/fcp/Beijing_Zang/sub10277
Shape: (128, 179, 174)
22.0
Processing FCP type dataset
Processing 767
/data-ixi-fcp/fcp/Beijing_Zang/sub10869
Shape: (128, 175, 168)
23.0
Processing FCP type dataset
Processing 768
/data-ixi-fcp/fcp/Beijing_Zang/sub10973
Shape: (128, 197, 205)
21.0
Processing FCP type dataset
Processing 769
/data-ixi-fcp/fcp/Beijing_Zang/sub11072
Shape: (128, 194, 193)
25.0
Processing FCP type dataset
Processing 770
/data-ixi-fcp/fcp/Beijing_Zang/sub11344
S

Shape: (128, 208, 185)
20.0
Processing FCP type dataset
Processing 838
/data-ixi-fcp/fcp/Beijing_Zang/sub44573
Shape: (128, 191, 155)
19.0
Processing FCP type dataset
Processing 839
/data-ixi-fcp/fcp/Beijing_Zang/sub45552
Shape: (128, 183, 209)
18.0
Processing FCP type dataset
Processing 840
/data-ixi-fcp/fcp/Beijing_Zang/sub46058
Shape: (128, 210, 216)
25.0
Processing FCP type dataset
Processing 841
/data-ixi-fcp/fcp/Beijing_Zang/sub46259
Shape: (128, 198, 179)
19.0
Processing FCP type dataset
Processing 842
/data-ixi-fcp/fcp/Beijing_Zang/sub46541
Shape: (128, 193, 194)
21.0
Processing FCP type dataset
Processing 843
/data-ixi-fcp/fcp/Beijing_Zang/sub48501
Shape: (128, 180, 193)
23.0
Processing FCP type dataset
Processing 844
/data-ixi-fcp/fcp/Beijing_Zang/sub48563
Shape: (128, 193, 200)
20.0
Processing FCP type dataset
Processing 845
/data-ixi-fcp/fcp/Beijing_Zang/sub48676
Shape: (128, 186, 180)
21.0
Processing FCP type dataset
Processing 846
/data-ixi-fcp/fcp/Beijing_Zang/sub49782
S

Shape: (128, 179, 180)
20.0
Processing FCP type dataset
Processing 914
/data-ixi-fcp/fcp/Beijing_Zang/sub83728
Shape: (128, 198, 196)
21.0
Processing FCP type dataset
Processing 915
/data-ixi-fcp/fcp/Beijing_Zang/sub85030
Shape: (128, 199, 181)
22.0
Processing FCP type dataset
Processing 916
/data-ixi-fcp/fcp/Beijing_Zang/sub85543
Shape: (128, 187, 173)
20.0
Processing FCP type dataset
Processing 917
/data-ixi-fcp/fcp/Beijing_Zang/sub85818
Shape: (176, 238, 200)
23.0
Processing FCP type dataset
Processing 918
/data-ixi-fcp/fcp/Beijing_Zang/sub86114
Shape: (128, 180, 159)
22.0
Processing FCP type dataset
Processing 919
/data-ixi-fcp/fcp/Beijing_Zang/sub87089
Shape: (128, 189, 163)
24.0
Processing FCP type dataset
Processing 920
/data-ixi-fcp/fcp/Beijing_Zang/sub87776
Shape: (128, 180, 200)
21.0
Processing FCP type dataset
Processing 921
/data-ixi-fcp/fcp/Beijing_Zang/sub88306
Shape: (128, 185, 185)
19.0
Processing FCP type dataset
Processing 922
/data-ixi-fcp/fcp/Beijing_Zang/sub88947
S

Shape: (155, 209, 149)
23.0
Processing FCP type dataset
Processing 988
/data-ixi-fcp/fcp/Bangor/sub81464
Shape: (170, 202, 161)
38.0
Processing FCP type dataset
Processing 989
/data-ixi-fcp/fcp/Bangor/sub82625
Shape: (164, 212, 177)
21.0
Processing FCP type dataset
Processing 990
/data-ixi-fcp/fcp/Bangor/sub87568
Shape: (155, 210, 154)
21.0
Processing FCP type dataset
Processing 991
/data-ixi-fcp/fcp/Bangor/sub91556
Shape: (162, 211, 179)
22.0
Processing FCP type dataset
Processing 992
/data-ixi-fcp/fcp/NewYork_b/sub05669
Shape: (128, 256, 256)
38.0
Processing FCP type dataset
Processing 993
/data-ixi-fcp/fcp/NewYork_b/sub06021
Shape: (128, 256, 256)
45.0
Processing FCP type dataset
Processing 994
/data-ixi-fcp/fcp/NewYork_b/sub27813
Shape: (128, 256, 256)
20.0
Processing FCP type dataset
Processing 995
/data-ixi-fcp/fcp/NewYork_b/sub28470
Shape: (128, 256, 256)
46.0
Processing FCP type dataset
Processing 996
/data-ixi-fcp/fcp/NewYork_b/sub31331
Shape: (128, 256, 256)
19.0
Processing F

In [4]:
print('Total data: ', len(df))
print('Available Data:', len(skullstripped_images_adni))
print('Unavailable Data:', len(files_not_found)) 

Total data:  1034
Available Data: 1033
Unavailable Data: 1


In [16]:
def process_dimension(dimension, image):
    d = image.shape[dimension]
    new_image = None
    if d < 224:
        remaining_dimensions = 224 - d
        if remaining_dimensions % 2 == 0:
            a = int(remaining_dimensions / 2)
            pad_width = ((0,0),(a, a)) if dimension == 1 else ((a,a),(0,0))
            new_image = np.pad(image, pad_width=pad_width, mode="constant", constant_values=0)
        else:
            a = int(remaining_dimensions / 2)
            b = a + 1
            pad_width = ((0,0),(a, b)) if dimension == 1 else ((a,b),(0,0))
            new_image = np.pad(image, pad_width=pad_width, mode="constant", constant_values=0)
    elif d > 224:
        extra_dimensions = d - 224
        if extra_dimensions % 2 == 0:
            a = int(extra_dimensions / 2)
            new_image = image[a:-a,:] if dimension == 0 else image[:, a:-a]
        else:
            a = int(extra_dimensions / 2)
            b = a + 1
            new_image = image[a:-b,:] if dimension == 0 else image[:, a:-b]
    else:
        print('here')
        return image
    return new_image

In [18]:
import torch

new_img_list = []
for i, skullstipped_image_adni in enumerate(skullstripped_images_adni):
    new_img_processed = np.zeros((3,224,224))
    x, y, z = skullstipped_image_adni.shape
    
    new_img = skullstipped_image_adni[int(x/2),:,:]
    new_img = process_dimension(0, new_img)
    new_img = process_dimension(1, new_img)
    new_img_processed[0] = new_img
    
    new_img = skullstipped_image_adni[:,int(y/2),:]
    new_img = process_dimension(0, new_img)
    new_img = process_dimension(1, new_img)
    new_img_processed[1] = new_img
    
    new_img = skullstipped_image_adni[:,:,int(z/2)]
    new_img = process_dimension(0, new_img)
    new_img = process_dimension(1, new_img)
    new_img_processed[2] = new_img
    print(f'Processed {i}, shape: {new_img_processed.shape}')
    new_img_list.append(new_img_processed)

x = np.asarray(new_img_list)
x = torch.Tensor(x).type(torch.float32)
torch.save(x, output_data)

Processed 0, shape: (3, 224, 224)
Processed 1, shape: (3, 224, 224)
Processed 2, shape: (3, 224, 224)
Processed 3, shape: (3, 224, 224)
Processed 4, shape: (3, 224, 224)
Processed 5, shape: (3, 224, 224)
Processed 6, shape: (3, 224, 224)
Processed 7, shape: (3, 224, 224)
Processed 8, shape: (3, 224, 224)
Processed 9, shape: (3, 224, 224)
Processed 10, shape: (3, 224, 224)
Processed 11, shape: (3, 224, 224)
Processed 12, shape: (3, 224, 224)
Processed 13, shape: (3, 224, 224)
Processed 14, shape: (3, 224, 224)
Processed 15, shape: (3, 224, 224)
Processed 16, shape: (3, 224, 224)
Processed 17, shape: (3, 224, 224)
Processed 18, shape: (3, 224, 224)
Processed 19, shape: (3, 224, 224)
Processed 20, shape: (3, 224, 224)
Processed 21, shape: (3, 224, 224)
Processed 22, shape: (3, 224, 224)
Processed 23, shape: (3, 224, 224)
Processed 24, shape: (3, 224, 224)
Processed 25, shape: (3, 224, 224)
Processed 26, shape: (3, 224, 224)
Processed 27, shape: (3, 224, 224)
Processed 28, shape: (3, 224, 

Processed 281, shape: (3, 224, 224)
Processed 282, shape: (3, 224, 224)
Processed 283, shape: (3, 224, 224)
Processed 284, shape: (3, 224, 224)
Processed 285, shape: (3, 224, 224)
Processed 286, shape: (3, 224, 224)
Processed 287, shape: (3, 224, 224)
Processed 288, shape: (3, 224, 224)
Processed 289, shape: (3, 224, 224)
Processed 290, shape: (3, 224, 224)
Processed 291, shape: (3, 224, 224)
Processed 292, shape: (3, 224, 224)
Processed 293, shape: (3, 224, 224)
Processed 294, shape: (3, 224, 224)
Processed 295, shape: (3, 224, 224)
Processed 296, shape: (3, 224, 224)
Processed 297, shape: (3, 224, 224)
Processed 298, shape: (3, 224, 224)
Processed 299, shape: (3, 224, 224)
Processed 300, shape: (3, 224, 224)
Processed 301, shape: (3, 224, 224)
Processed 302, shape: (3, 224, 224)
Processed 303, shape: (3, 224, 224)
Processed 304, shape: (3, 224, 224)
Processed 305, shape: (3, 224, 224)
Processed 306, shape: (3, 224, 224)
Processed 307, shape: (3, 224, 224)
Processed 308, shape: (3, 22

Processed 549, shape: (3, 224, 224)
Processed 550, shape: (3, 224, 224)
Processed 551, shape: (3, 224, 224)
Processed 552, shape: (3, 224, 224)
Processed 553, shape: (3, 224, 224)
Processed 554, shape: (3, 224, 224)
Processed 555, shape: (3, 224, 224)
Processed 556, shape: (3, 224, 224)
Processed 557, shape: (3, 224, 224)
Processed 558, shape: (3, 224, 224)
Processed 559, shape: (3, 224, 224)
Processed 560, shape: (3, 224, 224)
Processed 561, shape: (3, 224, 224)
Processed 562, shape: (3, 224, 224)
Processed 563, shape: (3, 224, 224)
Processed 564, shape: (3, 224, 224)
Processed 565, shape: (3, 224, 224)
Processed 566, shape: (3, 224, 224)
Processed 567, shape: (3, 224, 224)
Processed 568, shape: (3, 224, 224)
Processed 569, shape: (3, 224, 224)
Processed 570, shape: (3, 224, 224)
Processed 571, shape: (3, 224, 224)
Processed 572, shape: (3, 224, 224)
Processed 573, shape: (3, 224, 224)
Processed 574, shape: (3, 224, 224)
Processed 575, shape: (3, 224, 224)
Processed 576, shape: (3, 22

Processed 855, shape: (3, 224, 224)
Processed 856, shape: (3, 224, 224)
Processed 857, shape: (3, 224, 224)
Processed 858, shape: (3, 224, 224)
Processed 859, shape: (3, 224, 224)
Processed 860, shape: (3, 224, 224)
Processed 861, shape: (3, 224, 224)
Processed 862, shape: (3, 224, 224)
Processed 863, shape: (3, 224, 224)
Processed 864, shape: (3, 224, 224)
Processed 865, shape: (3, 224, 224)
Processed 866, shape: (3, 224, 224)
Processed 867, shape: (3, 224, 224)
Processed 868, shape: (3, 224, 224)
Processed 869, shape: (3, 224, 224)
Processed 870, shape: (3, 224, 224)
Processed 871, shape: (3, 224, 224)
Processed 872, shape: (3, 224, 224)
Processed 873, shape: (3, 224, 224)
Processed 874, shape: (3, 224, 224)
Processed 875, shape: (3, 224, 224)
Processed 876, shape: (3, 224, 224)
Processed 877, shape: (3, 224, 224)
Processed 878, shape: (3, 224, 224)
Processed 879, shape: (3, 224, 224)
Processed 880, shape: (3, 224, 224)
Processed 881, shape: (3, 224, 224)
Processed 882, shape: (3, 22

In [19]:
y = np.asarray(labels)
y = torch.Tensor(y).type(torch.float32)
torch.save(y, output_labels)